#### Test:

In [1]:
import dataset
from xai_cola import data,ml_model,counterfactual_explainer,cola,counterfactual_refiner
from xai_cola.cola_explainer import COunterfactualwithLimitedActions

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
dataset1 =dataset.GermanCreditDataset()
df1 = dataset1.get_dataframe()
df1.head(2)

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,1,2,1,0,1,1169,6,5,0
1,22,0,2,1,1,2,5951,48,5,1


In [4]:
# dataset2 = dataset.HotelBookingsDataset()
# df2 = dataset2.get_dataframe()
# df2.head(2)

##### Configuration

In [5]:
data1 = data.PandasData(data=df1, target_name="Risk")

In [6]:
import joblib

In [7]:
lgbmcClassifier = joblib.load('LGBMClassifier.pkl')
print(f'----LGBMClassifier.pkl model has been loaded----')

----LGBMClassifier.pkl model has been loaded----


In [8]:
ml_model1 = ml_model.Model(model=lgbmcClassifier, backend='sklearn')  

In [9]:
counterfactual_explainer1 = counterfactual_explainer.DiCE(ml_model=ml_model1, data=data1, sample_num=4)  #will add more input params according to the dice_ml algorithm 

---- predictions have been made----


100%|██████████| 4/4 [00:00<00:00,  8.05it/s]


---- predictions have been made----


100%|██████████| 4/4 [00:00<00:00,  7.20it/s]


In [10]:
x_factual, x_counterfactual = counterfactual_explainer1.generate_counterfactuals()   #numpy类型

---- predictions have been made----


100%|██████████| 4/4 [00:00<00:00,  8.58it/s]


##### *choose policy and make limitation


In [11]:
ot_pshap_max = COunterfactualwithLimitedActions(data=data1, ml_model=ml_model1, explainer=counterfactual_explainer1, limited_actions=10, matcher="ot", attributor="pshap", Avalues_method="max")

In [12]:
factual, ce, ace = ot_pshap_max.generate_results()

---- predictions have been made----
---- predictions have been made----
---- predictions have been made----


In [13]:
factual

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,33,0,2,1,0,2,1042,18,1,1
1,35,0,2,1,0,1,1381,24,1,1
2,34,1,1,1,1,1,11998,30,6,1
3,32,1,2,1,1,0,1530,18,1,1


In [14]:
ce

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,34,1,1,1,1,0,11998,30,6,0
1,32,1,2,1,1,0,1530,31,7,0
2,33,0,3,1,0,2,1042,18,0,0
3,73,0,2,1,0,1,1381,5,1,0


In [15]:
ace

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,33,0,3,1,0,2,1042,18,0,0
1,73,0,2,1,0,1,1381,5,1,0
2,34,1,1,1,1,0,11998,30,6,0
3,32,1,2,1,1,0,1530,31,7,0
